In [1]:
from tqdm import tqdm # Barra de progreso
import transformers

In [2]:
#Cambiar nombre modelo
# model_name = "./saved_model/"
model_name = "finiteautomata/beto-sentiment-analysis"
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
from transformers import pipeline
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [4]:
import pandas

archivo = "./data/PuertoMontt_chile_2021-07-01_2021-12-31_FILTRADO.csv"
dataset = pandas.read_csv(archivo)
dataset

,id_news,title,date
0,21443510.0,Para series menores de Puerto Varas y Puerto M...,2021-08-13
1,21469264.0,Puerto Montt: Hoy comienza el ciclo de concier...,2021-12-14
2,21940337.0,Más de 2500 rezagados fueron vacunados este fi...,2021-07-25
3,21469757.0,Así fue la marcha por el 18 de Octubre en Puer...,2021-10-19
4,47390948.0,De los artistas José Reyes y Gastón Ampuero: H...,2021-11-05
...,...,...,...
2703,21384707.0,¡Castro te quiero ver de pie…!,2021-12-10
2704,21391930.0,Víctor Hugo Catalán – poeta y narrador,2021-08-29
2705,21378371.0,Jordi Valenzuela Muñoz: una apuesta joven para...,2021-10-16
2706,47522784.0,"""El cambio climático puede tener grandes efect...",2021-08-24


In [5]:
dataset['title_label'] = ""
dataset['title_score'] = ""

for index, row in tqdm(dataset.iterrows(), desc='Análisis de sentimiento', total=dataset.shape[0]):
    
    # Analizamos su sentimiento en inglés
    sentiment_value = nlp(row['title'])
    
    # Insertamos en dataframe
    dataset.at[index, "title_label"] = sentiment_value[0].get('label')
    dataset.at[index, "title_score"] = sentiment_value[0].get('score')

Análisis de sentimiento: 100%|██████████| 2708/2708 [07:08<00:00,  6.32it/s]


In [20]:
doc=560
print(dataset['title'][doc])
print(dataset['title_label'][doc])
print(dataset['title_score'][doc])

Puerto Montt quedó como único líder de la Primera B
NEG
0.9735963940620422


In [18]:
dataset_negativos = dataset.where(dataset['title_label'] == 'NEG').dropna()
dataset_positivos = dataset.where(dataset['title_label'] == 'POS').dropna()

In [17]:
dataset_positivos

,id_news,title,date,title_label,title_score
0,21443510.0,Para series menores de Puerto Varas y Puerto M...,2021-08-13,POS,0.999743
1,21469264.0,Puerto Montt: Hoy comienza el ciclo de concier...,2021-12-14,POS,0.999865
2,21940337.0,Más de 2500 rezagados fueron vacunados este fi...,2021-07-25,POS,0.999808
3,21469757.0,Así fue la marcha por el 18 de Octubre en Puer...,2021-10-19,POS,0.99976
7,21425546.0,En la Sede UACh se realizará XXII Congreso Nac...,2021-12-03,POS,0.993462
...,...,...,...,...,...
2703,21384707.0,¡Castro te quiero ver de pie…!,2021-12-10,POS,0.999699
2704,21391930.0,Víctor Hugo Catalán – poeta y narrador,2021-08-29,POS,0.999865
2705,21378371.0,Jordi Valenzuela Muñoz: una apuesta joven para...,2021-10-16,POS,0.999873
2706,47522784.0,"""El cambio climático puede tener grandes efect...",2021-08-24,POS,0.9996


In [19]:
dataset_negativos.to_csv('./data/PuertoMontt_chile_2021-07-01_2021-12-31_NEGATIVOS_BETO.csv', index=False)
dataset_positivos.to_csv('./data/PuertoMontt_chile_2021-07-01_2021-12-31_POSITIVOS_BETO.csv', index=False)

In [21]:
#Tasa de noticias negativas
len(dataset_negativos)/len(dataset)

0.2710487444608567

In [22]:
#Tasa de noticias positivas
len(dataset_positivos)/len(dataset)

0.09711964549483014